# Weather Dataset Preprocessing and feature Engineering

In [1]:
%AddJar file:///home/jovyan/work/apps/Emiasd-Flight-Data-Analysis.jar

Starting download from file:///home/jovyan/work/apps/Emiasd-Flight-Data-Analysis.jar
Finished download of Emiasd-Flight-Data-Analysis.jar
Using cached version of Emiasd-Flight-Data-Analysis.jar


In [2]:
import org.apache.spark.sql.SparkSession
import com.flightdelay.config.{AppConfiguration, ConfigurationLoader}
import com.flightdelay.data.loaders.FlightDataLoader

//Env Configuration
val args: Array[String] = Array("jupyter")

val spark = SparkSession.builder()
  .config(sc.getConf)
  .getOrCreate()

// Rendre la session Spark implicite
implicit val session = spark
implicit val configuration: AppConfiguration = ConfigurationLoader.loadConfiguration(args)

val rawFlightsPath = s"${configuration.common.output.basePath}/common/data/raw_flights.parquet"
val flightsDF = spark.read.parquet(rawFlightsPath)

args = Array(jupyter)
spark = org.apache.spark.sql.SparkSession@1eabb303
session = org.apache.spark.sql.SparkSession@1eabb303
configuration = AppConfiguration(local,CommonConfig(42,DataConfig(/home/jovyan/work/data,FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Flights/201201.csv),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Weather/201201hourly.txt),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/wban_airport_timezone.csv)),OutputConfig(/home/jovyan/work/output,FileConfig(/home/jovyan/work/output/data),FileCo...


AppConfiguration(local,CommonConfig(42,DataConfig(/home/jovyan/work/data,FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Flights/201201.csv),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/Weather/201201hourly.txt),FileConfig(/home/jovyan/work/data/FLIGHT-3Y/wban_airport_timezone.csv)),OutputConfig(/home/jovyan/work/output,FileConfig(/home/jovyan/work/output/data),FileCo...

In [3]:
flightsDF.count()

18286055

In [4]:
flightsDF.printSchema

root
 |-- FL_DATE: date (nullable = true)
 |-- OP_CARRIER_AIRLINE_ID: integer (nullable = true)
 |-- OP_CARRIER_FL_NUM: integer (nullable = true)
 |-- ORIGIN_AIRPORT_ID: integer (nullable = true)
 |-- DEST_AIRPORT_ID: integer (nullable = true)
 |-- CRS_DEP_TIME: integer (nullable = true)
 |-- ARR_DELAY_NEW: double (nullable = true)
 |-- CANCELLED: integer (nullable = true)
 |-- DIVERTED: integer (nullable = true)
 |-- CRS_ELAPSED_TIME: double (nullable = true)
 |-- WEATHER_DELAY: double (nullable = true)
 |-- NAS_DELAY: double (nullable = true)



In [13]:
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.ml.feature.{StringIndexer, OneHotEncoder, VectorAssembler}
import scala.util.Try

In [24]:
flightsDF.count()

6096762

In [29]:
flightsDF.show(100)

+----------+---------------------+-----------------+-----------------+---------------+------------+-------------+---------+--------+----------------+-------------+---------+
|   FL_DATE|OP_CARRIER_AIRLINE_ID|OP_CARRIER_FL_NUM|ORIGIN_AIRPORT_ID|DEST_AIRPORT_ID|CRS_DEP_TIME|ARR_DELAY_NEW|CANCELLED|DIVERTED|CRS_ELAPSED_TIME|WEATHER_DELAY|NAS_DELAY|
+----------+---------------------+-----------------+-----------------+---------------+------------+-------------+---------+--------+----------------+-------------+---------+
|2012-07-01|                19805|             1123|            10693|          13303|         600|          0.0|     NULL|    NULL|           130.0|         NULL|     NULL|
|2012-07-01|                19805|             1124|            13303|          12953|         950|          0.0|     NULL|    NULL|           175.0|         NULL|     NULL|
|2012-07-01|                19805|             1127|            10821|          11298|        1035|          0.0|     NULL|    NUL

In [30]:
flightsDF.select("CANCELLED").distinct().show()

+---------+
|CANCELLED|
+---------+
|     NULL|
+---------+



In [31]:
flightsDF.select("DIVERTED").distinct().show()

+--------+
|DIVERTED|
+--------+
|    NULL|
+--------+



## 1. OP_CARRIER_AIRLINE_ID

In [8]:
flightsDF.select("OP_CARRIER_AIRLINE_ID").distinct().count()

15

In [7]:
flightsDF.select("OP_CARRIER_AIRLINE_ID").distinct().show(20)

+---------------------+
|OP_CARRIER_AIRLINE_ID|
+---------------------+
|                20398|
|                19690|
|                20409|
|                20378|
|                19790|
|                20304|
|                20436|
|                19393|
|                20355|
|                19805|
|                19930|
|                20366|
|                19977|
|                21171|
|                20437|
+---------------------+



In [11]:
flightsDF.select("OP_CARRIER_FL_NUM").distinct().count()

6670

In [10]:
import org.apache.spark.sql.functions._

flightsDF
  .groupBy("OP_CARRIER_AIRLINE_ID")
  .agg(countDistinct("OP_CARRIER_FL_NUM").as("nb_flights_distinct"))
  .orderBy(desc("nb_flights_distinct"))
  .show(50, false)

+---------------------+-------------------+
|OP_CARRIER_AIRLINE_ID|nb_flights_distinct|
+---------------------+-------------------+
|19393                |4946               |
|19790                |2100               |
|19805                |2056               |
|20366                |1961               |
|20437                |1750               |
|20398                |1703               |
|20355                |1671               |
|20304                |1664               |
|19977                |1580               |
|20409                |1028               |
|20436                |649                |
|19930                |618                |
|20378                |434                |
|19690                |318                |
|21171                |254                |
+---------------------+-------------------+



In [9]:
import org.apache.spark.sql.functions._

flightsDF
  .groupBy("OP_CARRIER_AIRLINE_ID", "OP_CARRIER_FL_NUM")
  .agg(count("*").as("nb_vols"))
  .orderBy(desc("nb_vols"))
  .show(50, false)

+---------------------+-----------------+-------+
|OP_CARRIER_AIRLINE_ID|OP_CARRIER_FL_NUM|nb_vols|
+---------------------+-----------------+-------+
|19930                |64               |1820   |
|19930                |65               |1815   |
|19930                |62               |1809   |
|19393                |24               |1580   |
|19930                |55               |1456   |
|19930                |61               |1448   |
|19930                |66               |1448   |
|19930                |67               |1416   |
|19393                |429              |1351   |
|19393                |47               |1327   |
|19393                |6                |1284   |
|19393                |1015             |1281   |
|19393                |910              |1251   |
|19393                |40               |1206   |
|19393                |1983             |1199   |
|19393                |132              |1152   |
|19393                |38               |1151   |


## 2. ORIGIN_AIRPORT_ID et DEST_AIRPORT_ID

In [14]:
flightsDF.select("ORIGIN_AIRPORT_ID").distinct().count()

312

In [15]:
flightsDF.select("DEST_AIRPORT_ID").distinct().count()

313

In [17]:
import org.apache.spark.sql.functions._

flightsDF
  .groupBy("ORIGIN_AIRPORT_ID")
  .agg(countDistinct("OP_CARRIER_FL_NUM").as("nb_flights_distinct"))
  .orderBy(desc("nb_flights_distinct"))
  .show(20, false)

+-----------------+-------------------+
|ORIGIN_AIRPORT_ID|nb_flights_distinct|
+-----------------+-------------------+
|11292            |4487               |
|12889            |3928               |
|12892            |3759               |
|14107            |3623               |
|10821            |3532               |
|13232            |3504               |
|10397            |3480               |
|13930            |3470               |
|14771            |2941               |
|13204            |2789               |
|14679            |2773               |
|15016            |2756               |
|13198            |2697               |
|10693            |2681               |
|11618            |2527               |
|12191            |2513               |
|11259            |2500               |
|12266            |2482               |
|11433            |2467               |
|11298            |2433               |
+-----------------+-------------------+
only showing top 20 rows



In [18]:
import org.apache.spark.sql.functions._

flightsDF
  .groupBy("DEST_AIRPORT_ID")
  .agg(countDistinct("OP_CARRIER_FL_NUM").as("nb_flights_distinct"))
  .orderBy(desc("nb_flights_distinct"))
  .show(20, false)

+---------------+-------------------+
|DEST_AIRPORT_ID|nb_flights_distinct|
+---------------+-------------------+
|11292          |4465               |
|12889          |3838               |
|12892          |3767               |
|14107          |3567               |
|13232          |3526               |
|10821          |3503               |
|13930          |3461               |
|10397          |3411               |
|14771          |2920               |
|13204          |2755               |
|15016          |2693               |
|14679          |2630               |
|13198          |2546               |
|12191          |2529               |
|10693          |2516               |
|11618          |2506               |
|11259          |2467               |
|12266          |2398               |
|11298          |2387               |
|11433          |2349               |
+---------------+-------------------+
only showing top 20 rows



## 3. CRS_ELAPSED_TIME

In [19]:
flightsDF.select("CRS_ELAPSED_TIME").distinct().count()

504

In [21]:
flightsDF.select("CRS_ELAPSED_TIME").distinct().show(20)

+----------------+
|CRS_ELAPSED_TIME|
+----------------+
|           305.0|
|           299.0|
|           170.0|
|           184.0|
|           147.0|
|           160.0|
|           169.0|
|            70.0|
|            67.0|
|           379.0|
|           311.0|
|           168.0|
|           206.0|
|            69.0|
|           389.0|
|           390.0|
|           365.0|
|           249.0|
|           142.0|
|           191.0|
+----------------+
only showing top 20 rows



In [23]:
flightsDF
  .select("CRS_ELAPSED_TIME")
  .summary("count", "min", "25%", "50%", "75%", "max", "mean", "stddev")
  .show()

+-------+-----------------+
|summary| CRS_ELAPSED_TIME|
+-------+-----------------+
|  count|          6096761|
|    min|              1.0|
|    25%|             81.0|
|    50%|            115.0|
|    75%|            165.0|
|    max|           1865.0|
|   mean|133.9327569179766|
| stddev|72.77188749215703|
+-------+-----------------+



In [32]:
import org.apache.spark.sql.functions._

// Récupérer les quantiles
val quantiles = flightsDF.stat.approxQuantile("CRS_ELAPSED_TIME", Array(0.25, 0.75), 0.0)
val q1 = quantiles(0)
val q3 = quantiles(1)
val iqr = q3 - q1

val lowerBound = q1 - 1.5 * iqr
val upperBound = q3 + 1.5 * iqr

println(s"Bornes acceptées : $lowerBound à $upperBound")

// Filtrer les données
val flightsNoOutliers = flightsDF.filter(
  col("CRS_ELAPSED_TIME") >= lowerBound && col("CRS_ELAPSED_TIME") <= upperBound
)

Bornes acceptées : -45.0 à 291.0


quantiles = Array(81.0, 165.0)
q1 = 81.0
q3 = 165.0
iqr = 84.0
lowerBound = -45.0
upperBound = 291.0
flightsNoOutliers = [FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 10 more fields]


[FL_DATE: date, OP_CARRIER_AIRLINE_ID: int ... 10 more fields]